In [1]:
from pybedtools import BedTool
import pandas as pd
import numpy as np
import shutil
import csv
import os

pd.set_option('display.max_columns',500)

outDir="./prepAnnotation"
alignmentDir=outDir+"/alt_alignments"

blatCols=["matches",
        "misMatches",
        "repMatches",
        "nCount",
        "qNumInsert",
        "qBaseInsert",
        "tNumInsert",
        "tBaseInsert",
        "strand",
        "qName",
        "qSize",
        "qStart",
        "qEnd",
        "tName",
        "tSize",
        "tStart",
        "tEnd",
        "blockCount",
        "blockSizes",
        "qStarts",
        "tStarts"]
intersectCols=["chrom",
                "source",
                "type",
                "start",
                "end",
                "score",
                "strand",
                "phase",
                "attributes",
                "chromK",
                "sourceK",
                "typeK",
                "startK",
                "endK",
                "scoreK",
                "strandK",
                "phaseK",
                "attributesK",
                "distance"]
samCols=['QNAME',
         'FLAG',
         'RNAME',
         'POS',
         'MAPQ',
         'CIGAR',
         'RNEXT',
         'PNEXT',
         'TLEN',
         'SEQ',
         'QUAL']

gff3Cols=["seqid","source","type","start","end","score","strand","phase","attributes"]

In [2]:
dfAll=pd.DataFrame([])
for psl in os.listdir(alignmentDir):
    if psl[-4:]==".psl":
        chrom=".".join(psl.split(".")[:-1])
        fp=alignmentDir+"/"+psl
        psl=pd.read_csv(fp,sep="\t",skiprows=5,names=blatCols)
        psl=psl[psl["tName"]==chrom].sort_values(by="matches",ascending=False).reset_index(drop=True).drop_duplicates(["qName"])#.drop(["repMatches","nCount","qBaseInsert","tNumInsert"],axis=1)
        dfAll=pd.concat([dfAll,psl],axis=0).reset_index(drop=True)
dfAll

,matches,misMatches,repMatches,nCount,qNumInsert,qBaseInsert,tNumInsert,tBaseInsert,strand,qName,qSize,qStart,qEnd,tName,tSize,tStart,tEnd,blockCount,blockSizes,qStarts,tStarts
0,4307,0,0,0,0,0,16,26820,-,ENST00000630761.2,4307,0,4307,chr21,46709983,46188445,46219572,17,"2790,79,171,81,66,106,97,150,51,72,57,28,98,11...","0,2790,2869,3040,3121,3187,3293,3390,3540,3591...","46188445,46191880,46194490,46195675,46196201,4..."
1,4121,0,0,0,0,0,3,18950,-,rna-NM_001162495.2-2,4121,0,4121,chr21,46709983,32790672,32813743,4,"3813,75,81,152,","0,3813,3888,3969,","32790672,32807641,32810573,32813591,"
2,4046,0,0,0,0,0,2,19025,-,rna-NM_019596.5-2,4046,0,4046,chr21,46709983,32790672,32813743,3,"3813,81,152,","0,3813,3894,","32790672,32810573,32813591,"
3,3965,0,0,0,0,0,1,19106,-,rna-NM_001162496.2-2,3965,0,3965,chr21,46709983,32790672,32813743,2,"3813,152,","0,3813,","32790672,32813591,"
4,3127,0,0,0,0,0,8,44397,-,rna-NR_040016.1-2,3127,0,3127,chr21,46709983,33432485,33480009,9,"317,84,2082,58,93,68,113,140,172,","0,317,401,2483,2541,2634,2702,2815,2955,","33432485,33439213,33448781,33455726,33460412,3..."
5,2453,0,0,0,0,0,3,24417,+,ENST00000575401.5,2453,0,2453,chr21,46709983,32772099,32798969,4,"209,123,90,2031,","0,209,332,422,","32772099,32784796,32788611,32796938,"
6,2429,0,0,0,0,0,0,0,+,rna-NR_149073.1-2,2429,0,2429,chr21,46709983,17611685,17614114,1,"2429,","0,","17611685,"
7,2375,0,0,0,0,0,6,28502,-,ENST00000573374.5,2375,0,2375,chr21,46709983,33449134,33480011,7,"1729,58,93,68,113,140,174,","0,1729,1787,1880,1948,2061,2201,","33449134,33455726,33460412,33465341,33467009,3..."
8,2368,0,0,0,0,0,17,29064,-,ENST00000630343.2,2368,0,2368,chr21,46709983,46188140,46219572,18,"681,170,79,171,81,66,106,97,150,51,72,57,28,98...","0,681,851,930,1101,1182,1248,1354,1451,1601,16...","46188140,46191065,46191880,46194490,46195675,4..."
9,2316,0,0,0,0,0,4,20996,-,rna-NM_001142854.1-2,2316,0,2316,chr21,46709983,46161147,46184459,5,"558,152,351,1151,104,","0,558,710,1061,2212,","46161147,46161915,46168307,46182623,46184355,"


In [3]:
# need to re-write the writeGFF function to make faster
includeCols=["matches",
                "misMatches",
                "repMatches",
                "nCount",
                "qNumInsert",
                "qBaseInsert",
                "tNumInsert",
                "tBaseInsert",
                "strand",
                "qName",
                "qSize",
                "qStart",
                "qEnd",
                "tName",
                "tSize",
                "tStart",
                "tEnd",
                "blockCount"]
splitCols=["blockSizes",
           "qStarts",
           "tStarts"]
# make unique id from index
dfAll["uid"]=dfAll.reset_index(drop=True).reset_index()["index"]

dfGFF=pd.DataFrame([])
for col in splitCols:
    tmp=pd.concat([pd.Series(row['uid'], row[col].split(','))              
                        for _, row in dfAll.iterrows()]).reset_index()
    tmp.columns=[col,"uid"]
    dfGFF=pd.concat([dfGFF,tmp],axis=1)
    
dfGFF=dfGFF[~((dfGFF["blockSizes"]==dfGFF["qStarts"])\
              &(dfGFF["qStarts"]==dfGFF["tStarts"])\
              &(dfGFF["tStarts"]==""))].reset_index(drop=True) # get rid of empty lines
dfGFF.columns=["blockSizes","uid1","qStarts","uid2","tStarts","uid"]
dfGFF.drop(["uid1","uid2"],axis=1,inplace=True)
dfGFF=dfGFF.merge(dfAll[["tName","strand","uid"]],on="uid",how="left")
dfGFF["phase"]="."
dfGFF["score"]="."
dfGFF["ref"]="ref"
dfGFF["type"]="exon"
dfGFF["attribute"]=dfGFF["uid"]
dfGFF["start"]=dfGFF['tStarts']
dfGFF['end']=dfGFF['start'].astype(int)+dfGFF["blockSizes"].astype(int)
dfGFF=dfGFF[['tName',\
             'ref',\
             'type',\
             'start',\
             'end',\
             'score',\
             'strand',\
             'phase',\
             'attribute']]
dfAll[["qName","uid"]].to_csv(outDir+"/blat_map.csv",index=False)

In [4]:
# we can try to write out a gff of the blocks for the gff compare

# first need to add a transcript feature for each exon group
dft=dfGFF.groupby(by='attribute').agg({"start":"min","end":"max","ref":"min","tName":"min","score":"min","strand":"min","phase":"min"}).reset_index()
dft['type']='transcript'
dft['attribute']='ID='+dft['attribute'].astype(str)
dfGFF['attribute']='Parent='+dfGFF['attribute'].astype(str)
dfGFF_f=pd.concat([dft[['tName','ref','type','start','end','score','strand','phase','attribute']],dfGFF])

dfGFF_f["parent"] = np.where(dfGFF_f["type"]=="exon",
                             dfGFF_f["attribute"].str.split("Parent=",expand=True)[1].str.split(";",expand=True)[0],
                             dfGFF_f["attribute"].str.split("ID=",expand=True)[1].str.split(";",expand=True)[0])

dfGFF_f["type"]=pd.Categorical(dfGFF_f["type"],categories=["transcript","exon"],ordered=True)
dfGFF_f=dfGFF_f.sort_values(by=["parent","type"]).reset_index(drop=True)

dfGFF_f[["tName","ref","type","start","end","score","strand","phase","attribute"]].to_csv(outDir+'/dfGFF_blat.gtf',sep="\t",index=False,header=False,quoting=csv.QUOTE_NONE)
dfGFF_f = dfGFF_f[["tName","ref","type","start","end","score","strand","phase","attribute"]]
# gffread -E dfGFF.gtf  -o- > dfGFF.gff

In [5]:
dfPrimExon = pd.read_csv("./prepAnnotation/prims_and_rest.gff",sep="\t",names=gff3Cols)
display(dfPrimExon.head())
annotation=BedTool.from_dataframe(dfPrimExon)

,seqid,source,type,start,end,score,strand,phase,attributes
0,chr7,HAVANA,transcript,127588345,127591705,.,+,.,ID=ENST00000000233.9;gene_id=ENSG00000004059.1...
1,chr7,HAVANA,exon,127588345,127588565,.,+,.,Parent=ENST00000000233.9
2,chr7,HAVANA,exon,127589083,127589163,.,+,.,Parent=ENST00000000233.9
3,chr7,HAVANA,exon,127589485,127589594,.,+,.,Parent=ENST00000000233.9
4,chr7,HAVANA,exon,127590066,127590137,.,+,.,Parent=ENST00000000233.9


In [6]:
## now to run bedtools on the entire thing
sites=BedTool.from_dataframe(dfGFF)
nearby=sites.intersect(annotation,wao=True)
dfIntersect=pd.read_table(nearby.fn,names=intersectCols,index_col=False)
dfIntersect

/home/sparrow/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  after removing the cwd from sys.path.


,chrom,source,type,start,end,score,strand,phase,attributes,chromK,sourceK,typeK,startK,endK,scoreK,strandK,phaseK,attributesK,distance
0,chr21,ref,exon,46188445,46191235,.,-,.,Parent=0,chr21,HAVANA,exon,46188141,46188821,.,-,.,Parent=ENST00000356396.8,377
1,chr21,ref,exon,46188445,46191235,.,-,.,Parent=0,chr21,HAVANA,exon,46191066,46191235,.,-,.,Parent=ENST00000356396.8,170
2,chr21,ref,exon,46188445,46191235,.,-,.,Parent=0,chr21,HAVANA,CDS,46191104,46191235,.,-,0,Parent=ENST00000356396.8,132
3,chr21,ref,exon,46188445,46191235,.,-,.,Parent=0,chr21,HAVANA,exon,46188446,46191235,.,-,.,Parent=ENST00000397728.7,2790
4,chr21,ref,exon,46188445,46191235,.,-,.,Parent=0,chr21,HAVANA,CDS,46191104,46191235,.,-,0,Parent=ENST00000397728.7,132
5,chr21,ref,exon,46188445,46191235,.,-,.,Parent=0,chr21,ENSEMBL,exon,46189124,46191235,.,-,.,Parent=ENST00000457828.6,2112
6,chr21,ref,exon,46188445,46191235,.,-,.,Parent=0,chr21,ENSEMBL,CDS,46191104,46191235,.,-,0,Parent=ENST00000457828.6,132
7,chr21,ref,exon,46188445,46191235,.,-,.,Parent=0,chr21,BestRefSeq,exon,46189124,46191235,.,-,.,Parent=rna-NM_001145436.1,2112
8,chr21,ref,exon,46188445,46191235,.,-,.,Parent=0,chr21,BestRefSeq,CDS,46191104,46191235,.,-,0,Parent=rna-NM_001145436.1,132
9,chr21,ref,exon,46188445,46191235,.,-,.,Parent=0,chr21,HAVANA,transcript,46188141,46228822,.,-,.,ID=ENST00000356396.8;gene_id=ENSG00000160285.1...,2791


In [7]:
def writeGFF(row):
    percentOverlaps=[]
    percentOverlaps2=[]
    percentOverlaps3=[]
    wpo=[]
    wpo2=[]
    wpoL=1
    wpo2L=1
    
    if row.distance==0:
        percentOverlaps.append(0)
        percentOverlaps2.append(0)
        wpo2L=wpo2L+float(int(row.end)-int(row.start))
    else:
        po=round(float(row.distance)/float(int(row.endK)-int(row.startK)),4)
        if po>1.0:
            po=1.0
        percentOverlaps.append(po)
        percentOverlaps2.append(round(float(row.distance)/float(int(row.end)-int(row.start)),4))
        percentOverlaps3.append(".".join(row.attributesK.split("Parent=")[-1].split(".")[:-1]))

    return [";".join([str(x) for x in percentOverlaps]),";".join([str(x) for x in percentOverlaps2]),";".join(list(set(percentOverlaps3)))]

dfIntersect["po_prim"]=np.nan
dfIntersect["po_alt"]=np.nan
dfIntersect["ann"]=np.nan
dfIntersect[["po_prim","po_alt","ann"]]=pd.DataFrame([x for x in dfIntersect.apply(lambda row: writeGFF(row),axis=1)])
dfIntersect.to_csv(outDir+"/finalMappingOverlap_exon_2.csv",index=False)
dfIntersect['po_prim'].astype(float)
dfIntersect['po_alt'].astype(float)
dfIntersect["lid"] = dfIntersect["attributes"].str.split("Parent=",expand=True)[1].str.split(";",expand=True)[0].astype(int)
dfIntersect=dfIntersect.merge(dfAll[['uid','blockCount']],how="left",left_on="lid",right_on="uid")
dfIntersect

,chrom,source,type,start,end,score,strand,phase,attributes,chromK,sourceK,typeK,startK,endK,scoreK,strandK,phaseK,attributesK,distance,po_prim,po_alt,ann,lid,uid,blockCount
0,chr21,ref,exon,46188445,46191235,.,-,.,Parent=0,chr21,HAVANA,exon,46188141,46188821,.,-,.,Parent=ENST00000356396.8,377,0.5544,0.1351,ENST00000356396,0,0,17
1,chr21,ref,exon,46188445,46191235,.,-,.,Parent=0,chr21,HAVANA,exon,46191066,46191235,.,-,.,Parent=ENST00000356396.8,170,1.0,0.0609,ENST00000356396,0,0,17
2,chr21,ref,exon,46188445,46191235,.,-,.,Parent=0,chr21,HAVANA,CDS,46191104,46191235,.,-,0,Parent=ENST00000356396.8,132,1.0,0.0473,ENST00000356396,0,0,17
3,chr21,ref,exon,46188445,46191235,.,-,.,Parent=0,chr21,HAVANA,exon,46188446,46191235,.,-,.,Parent=ENST00000397728.7,2790,1.0,1.0,ENST00000397728,0,0,17
4,chr21,ref,exon,46188445,46191235,.,-,.,Parent=0,chr21,HAVANA,CDS,46191104,46191235,.,-,0,Parent=ENST00000397728.7,132,1.0,0.0473,ENST00000397728,0,0,17
5,chr21,ref,exon,46188445,46191235,.,-,.,Parent=0,chr21,ENSEMBL,exon,46189124,46191235,.,-,.,Parent=ENST00000457828.6,2112,1.0,0.757,ENST00000457828,0,0,17
6,chr21,ref,exon,46188445,46191235,.,-,.,Parent=0,chr21,ENSEMBL,CDS,46191104,46191235,.,-,0,Parent=ENST00000457828.6,132,1.0,0.0473,ENST00000457828,0,0,17
7,chr21,ref,exon,46188445,46191235,.,-,.,Parent=0,chr21,BestRefSeq,exon,46189124,46191235,.,-,.,Parent=rna-NM_001145436.1,2112,1.0,0.757,rna-NM_001145436,0,0,17
8,chr21,ref,exon,46188445,46191235,.,-,.,Parent=0,chr21,BestRefSeq,CDS,46191104,46191235,.,-,0,Parent=rna-NM_001145436.1,132,1.0,0.0473,rna-NM_001145436,0,0,17
9,chr21,ref,exon,46188445,46191235,.,-,.,Parent=0,chr21,HAVANA,transcript,46188141,46228822,.,-,.,ID=ENST00000356396.8;gene_id=ENSG00000160285.1...,2791,0.0686,1.0004,ID=ENST00000356396.8;gene_id=ENSG00000160285.1...,0,0,17


In [8]:
# let's try to write cases manually

# first we shall extract all the ones that match perfectly - any such transcripts can be discarded, since they already exist on the primary scaffolds
# additionally, a threshold can be set for each PO lower than 1.0 - can be discussed
dfPerfect=dfIntersect[(dfIntersect["po_prim"]=="1.0")&(dfIntersect["po_alt"]=="1.0")]# of these we need to figure out how many constitute full transcripts
dfExists=dfPerfect[['attributesK','po_prim','blockCount','start','end','strand','attributes']].groupby(by=['attributesK','start','end','strand','attributes']).agg({'po_prim':'count','blockCount':'min'}).reset_index()
dfExists[dfExists['po_prim']==dfExists['blockCount']]
# tofind such transcripts we need to have an expected exon count per transcript and the transcript count for each corresponding primary transcript

,attributesK,start,end,strand,attributes,po_prim,blockCount
0,ID=ENST00000240619.2;gene_id=ENSG00000121318.2...,10825316,10826358,-,Parent=3719,1,1
1,ID=ENST00000240619.2;gene_id=ENSG00000121318.2...,10825316,10826358,-,Parent=3720,1,1
2,ID=ENST00000240687.2;gene_id=ENSG00000121377.2...,10801531,10802627,-,Parent=3715,1,1
3,ID=ENST00000240687.2;gene_id=ENSG00000121377.2...,10801531,10802627,-,Parent=3716,1,1
4,ID=ENST00000240691.4;gene_id=ENSG00000121381.4...,10809093,10810168,-,Parent=3717,1,1
5,ID=ENST00000240691.4;gene_id=ENSG00000121381.4...,10809093,10810168,-,Parent=3718,1,1
6,ID=ENST00000302270.1;gene_id=ENSG00000172199.1...,56375623,56376553,+,Parent=3041,1,1
7,ID=ENST00000303039.3;gene_id=ENSG00000172487.4...,56360214,56361288,+,Parent=3015,1,1
8,ID=ENST00000303059.3;gene_id=ENSG00000172489.6...,56252199,56253222,+,Parent=3025,1,1
9,ID=ENST00000306271.4;gene_id=ENSG00000188581.8...,41040558,41041461,-,Parent=3515,1,1


In [46]:
dfPerfect

,chrom,source,type,start,end,score,strand,phase,attributes,chromK,sourceK,typeK,startK,endK,scoreK,strandK,phaseK,attributesK,distance,po_prim,po_alt,ann,lid,uid,blockCount
3,chr21,ref,exon,46188445,46191235,.,-,.,Parent=0,chr21,HAVANA,exon,46188446,46191235,.,-,.,Parent=ENST00000397728.7,2790,1.0,1.0,ENST00000397728,0,0,17
13,chr21,ref,exon,46191880,46191959,.,-,.,Parent=0,chr21,HAVANA,exon,46191881,46191959,.,-,.,Parent=ENST00000356396.8,79,1.0,1.0,ENST00000356396,0,0,17
14,chr21,ref,exon,46191880,46191959,.,-,.,Parent=0,chr21,HAVANA,CDS,46191881,46191959,.,-,1,Parent=ENST00000356396.8,79,1.0,1.0,ENST00000356396,0,0,17
15,chr21,ref,exon,46191880,46191959,.,-,.,Parent=0,chr21,HAVANA,exon,46191881,46191959,.,-,.,Parent=ENST00000397728.7,79,1.0,1.0,ENST00000397728,0,0,17
16,chr21,ref,exon,46191880,46191959,.,-,.,Parent=0,chr21,HAVANA,CDS,46191881,46191959,.,-,1,Parent=ENST00000397728.7,79,1.0,1.0,ENST00000397728,0,0,17
17,chr21,ref,exon,46191880,46191959,.,-,.,Parent=0,chr21,ENSEMBL,exon,46191881,46191959,.,-,.,Parent=ENST00000457828.6,79,1.0,1.0,ENST00000457828,0,0,17
18,chr21,ref,exon,46191880,46191959,.,-,.,Parent=0,chr21,ENSEMBL,CDS,46191881,46191959,.,-,1,Parent=ENST00000457828.6,79,1.0,1.0,ENST00000457828,0,0,17
19,chr21,ref,exon,46191880,46191959,.,-,.,Parent=0,chr21,BestRefSeq,exon,46191881,46191959,.,-,.,Parent=rna-NM_001145436.1,79,1.0,1.0,rna-NM_001145436,0,0,17
20,chr21,ref,exon,46191880,46191959,.,-,.,Parent=0,chr21,BestRefSeq,CDS,46191881,46191959,.,-,1,Parent=rna-NM_001145436.1,79,1.0,1.0,rna-NM_001145436,0,0,17
25,chr21,ref,exon,46194490,46194661,.,-,.,Parent=0,chr21,HAVANA,exon,46194491,46194661,.,-,.,Parent=ENST00000356396.8,171,1.0,1.0,ENST00000356396,0,0,17


In [9]:
df=pd.read_csv(outDir+"/dfGFF_blat.gtf",sep="\t",names=gff3Cols)

df["id"]=df.attributes.str.split("ID=",expand=True)[1].str.split(";",expand=True)[0]
dfE=df[df["type"]=="exon"].reset_index(drop=True)
dfE["transID"]=dfE.attributes.str.split("Parent=",expand=True)[1].str.split(";",expand=True)[0]
transcriptSet=set(dfE["transID"])
dfT=df[df["id"].isin(transcriptSet)].reset_index(drop=True)
assert len(set(dfT["id"]))==len(transcriptSet)
dfT["transID"]=dfT["id"]

# here's the time to deal with inconsistencies
# first remove any enties that do not have a parent - those do not conform to the expected structure that we define
# however, this way we remove PSEUDOGENES, which may not be the desired effect - need to think more/discuss
dfE=dfE[dfE["transID"].isin(dfT["transID"])].reset_index(drop=True)
dfT["type"]="transcript"

dfE=dfE.merge(dfT[["transID"]],how="left",left_on="transID",right_on="transID")

# now need to concatenate all these three dataframes
dfM=pd.concat([dfT[["seqid","source","type","start","end","score","strand","phase","attributes","transID"]],dfE[["seqid","source","type","start","end","score","strand","phase","attributes","transID"]]],axis=0).reset_index(drop=True)
dfM["type"]=pd.Categorical(dfM["type"],categories=["transcript","exon"],ordered=True)
dfM=dfM.sort_values(by=["transID","type"]).reset_index(drop=True)
del dfE
del dfT
del df

dfM["start"]=dfM["start"].astype(int)+1
dfM["end"]=dfM["end"].astype(int)
# some pseudogenes are still present

dfM_g=dfM[dfM["type"].isin(["exon"])][["seqid","strand","type","start","end","transID"]].reset_index(drop=True)
dfM_g=dfM_g.sort_values(by=["transID","start"]).reset_index(drop=True)
dfM_g["ns"]=dfM_g.start.astype(str).shift(-1)
dfM_g["nid"]=dfM_g.transID.shift(-1)
dfM_g["ns"]=np.where(dfM_g["transID"]==dfM_g["nid"],dfM_g["ns"],"***")
dfM_g["ns"]=dfM_g.ns.astype(str)+"+"
dfM_g["ends"]=dfM_g.end.astype(str)+"+"
dfM_g=dfM_g[~(dfM_g['ns']=="***+")].reset_index(drop=True)
dfM_g=dfM_g.groupby(by="transID").agg({'start':'min',\
                                       'ns':'sum',\
                                       'ends':'sum',\
                                       'end':'max',\
                                       'strand':'min',\
                                       'seqid':'min',\
                                       'nid':'count'})
dfM_g.rename({"nid":'count'},axis=1,inplace=True)
dfM_g.reset_index(inplace=True)
dfM_g.columns=['transID','start_min',"ns",'end','end_max','strand','seqid','count']
dfM_g["ns"]=dfM_g["ns"].str.rstrip("***+")
dfM_g["end"]=dfM_g["end"].str.rstrip("***+")
dfM_g["chain"]=dfM_g['seqid']+":"+dfM_g['strand']+'@'+dfM_g['end']+'-'+dfM_g['ns']
dfM_g=dfM_g[['transID','start_min','end_max','chain','count']]

del transcriptSet
dfM=dfM[gff3Cols+["transID"]]
dfM

,seqid,source,type,start,end,score,strand,phase,attributes,transID
0,chr21,ref,transcript,46188446,46219572,.,-,.,ID=0,0
1,chr21,ref,exon,46188446,46191235,.,-,.,Parent=0,0
2,chr21,ref,exon,46191881,46191959,.,-,.,Parent=0,0
3,chr21,ref,exon,46194491,46194661,.,-,.,Parent=0,0
4,chr21,ref,exon,46195676,46195756,.,-,.,Parent=0,0
5,chr21,ref,exon,46196202,46196267,.,-,.,Parent=0,0
6,chr21,ref,exon,46205836,46205941,.,-,.,Parent=0,0
7,chr21,ref,exon,46206672,46206768,.,-,.,Parent=0,0
8,chr21,ref,exon,46207428,46207577,.,-,.,Parent=0,0
9,chr21,ref,exon,46208251,46208301,.,-,.,Parent=0,0


In [10]:
df=pd.read_csv("./prepAnnotation/prims_and_rest.gff",sep='\t',names=gff3Cols)
df=df[df["type"].isin(['transcript','exon'])].reset_index(drop=True)

df["id"]=df.attributes.str.split("ID=",expand=True)[1].str.split(";",expand=True)[0]
dfE=df[df["type"]=="exon"].reset_index(drop=True)
dfE["transID"]=dfE.attributes.str.split("Parent=",expand=True)[1].str.split(";",expand=True)[0]
transcriptSet=set(dfE["transID"])
dfT=df[df["id"].isin(transcriptSet)].reset_index(drop=True)
assert len(set(dfT["id"]))==len(transcriptSet)
dfT["transID"]=dfT["id"]

# here's the time to deal with inconsistencies
# first remove any enties that do not have a parent - those do not conform to the expected structure that we define
# however, this way we remove PSEUDOGENES, which may not be the desired effect - need to think more/discuss
dfE=dfE[dfE["transID"].isin(dfT["transID"])].reset_index(drop=True)
dfT["type"]="transcript"

dfE=dfE.merge(dfT[["transID"]],how="left",left_on="transID",right_on="transID")

# now need to concatenate all these three dataframes
dfM_2=pd.concat([dfT[["seqid","source","type","start","end","score","strand","phase","attributes","transID"]],dfE[["seqid","source","type","start","end","score","strand","phase","attributes","transID"]]],axis=0).reset_index(drop=True)
dfM_2["type"]=pd.Categorical(dfM_2["type"],categories=["transcript","exon"],ordered=True)
dfM_2=dfM_2.sort_values(by=["transID","type"]).reset_index(drop=True)
del dfE
del dfT
del df

dfM_2["start"]=dfM_2["start"].astype(int)
dfM_2["end"]=dfM_2["end"].astype(int)
# some pseudogenes are still present

dfM_2_g=dfM_2[dfM_2["type"].isin(["exon"])][["seqid","strand","type","start","end","transID"]].reset_index(drop=True)
dfM_2_g=dfM_2_g.sort_values(by=["transID","start"]).reset_index(drop=True)
dfM_2_g["ns"]=dfM_2_g.start.astype(str).shift(-1)
dfM_2_g["nid"]=dfM_2_g.transID.shift(-1)
dfM_2_g["ns"]=np.where(dfM_2_g["transID"]==dfM_2_g["nid"],dfM_2_g["ns"],"***")
dfM_2_g["ns"]=dfM_2_g.ns.astype(str)+"+"
dfM_2_g["ends"]=dfM_2_g.end.astype(str)+"+"
dfM_2_g=dfM_2_g[~(dfM_2_g['ns']=="***+")].reset_index(drop=True)
dfM_2_g=dfM_2_g.groupby(by="transID").agg({'start':'min','ns':'sum','ends':'sum','end':'max','strand':'min','seqid':'min','nid':'count'})
dfM_2_g.rename({"nid":'count'},axis=1,inplace=True)
dfM_2_g.reset_index(inplace=True)
dfM_2_g.columns=['transID','start_min',"ns",'end','end_max','strand','seqid','count']
dfM_2_g["ns"]=dfM_2_g["ns"].str.rstrip("***+")
dfM_2_g["end"]=dfM_2_g["end"].str.rstrip("***+")
dfM_2_g["chain"]=dfM_2_g['seqid']+":"+dfM_2_g['strand']+'@'+dfM_2_g['end']+'-'+dfM_2_g['ns']
dfM_2_g=dfM_2_g[['transID','start_min','end_max','chain','count']]

del transcriptSet
dfM_2=dfM_2[gff3Cols+["transID"]]
dfM_2

,seqid,source,type,start,end,score,strand,phase,attributes,transID
0,chr7,HAVANA,transcript,127588345,127591705,.,+,.,ID=ENST00000000233.9;gene_id=ENSG00000004059.1...,ENST00000000233.9
1,chr7,HAVANA,exon,127588345,127588565,.,+,.,Parent=ENST00000000233.9,ENST00000000233.9
2,chr7,HAVANA,exon,127589083,127589163,.,+,.,Parent=ENST00000000233.9,ENST00000000233.9
3,chr7,HAVANA,exon,127589485,127589594,.,+,.,Parent=ENST00000000233.9,ENST00000000233.9
4,chr7,HAVANA,exon,127590066,127590137,.,+,.,Parent=ENST00000000233.9,ENST00000000233.9
5,chr7,HAVANA,exon,127590963,127591088,.,+,.,Parent=ENST00000000233.9,ENST00000000233.9
6,chr7,HAVANA,exon,127591213,127591705,.,+,.,Parent=ENST00000000233.9,ENST00000000233.9
7,chr12,HAVANA,transcript,8940365,8949955,.,-,.,ID=ENST00000000412.7;gene_id=ENSG00000003056.7...,ENST00000000412.7
8,chr12,HAVANA,exon,8940365,8941940,.,-,.,Parent=ENST00000000412.7,ENST00000000412.7
9,chr12,HAVANA,exon,8942416,8942542,.,-,.,Parent=ENST00000000412.7,ENST00000000412.7


In [11]:
if not os.path.exists("./prepAnnotation/res_blat"):
    os.mkdir("./prepAnnotation/res_blat")

# Step 1 - check if there are any that correspond to any intron chains entirely
#          these are the ones that already exist
intron_compatible=dfM_2_g.merge(dfM_g,how='inner',on='chain')
intron_compatible=intron_compatible[['transID_x','transID_y']]
intron_compatible.columns=['transID_prim','transID_alt']
intron_compatible.to_csv("./prepAnnotation/res_blat/intron_compatible.csv",index=False)

# step 2, check any single-exon transcripts that are contained within any other single-exon transcripts
singleDF_2=dfM_2[~(dfM_2["transID"].isin(dfM_2_g["transID"]))&(dfM_2['type']=='transcript')].reset_index(drop=True)
singleDF=dfM[~(dfM["transID"].isin(dfM_g["transID"]))&(dfM['type']=='transcript')].reset_index(drop=True)
singleDF=singleDF_2.merge(singleDF,on=['seqid','start','end','strand'],how="inner")
singleDF=singleDF[["transID_x","transID_y"]].reset_index(drop=True)
singleDF.columns=['transID_prim','transID_alt']
singleDF.to_csv("./prepAnnotation/res_blat/singleDF.csv",index=False)

# Step 2.5 we can now take the dataframes above, concatenate them and process
# these transcripts belong to genes that are guaranteed to exist on the primary scaffolds so we can move them
# and the rest of the sibling transcripts to the primary scaffolds
# and remove from the alternative scaffolds
perfectMatch=pd.concat([intron_compatible,singleDF])
mapDF=pd.read_csv(outDir+"/blat_map.csv")
mapDF['uid']=mapDF['uid'].astype(str)
perfectMatch=perfectMatch.merge(mapDF,how='left',left_on='transID_alt',right_on='uid')
perfectMatch.drop('uid',axis=1,inplace=True)
perfectMatch.columns=['transID_prim','transID_alt','transID_alt_2']
perfectMatch.to_csv("./prepAnnotation/res_blat/perfectMatch_blat.csv",index=False)
perfectMatch

,transID_prim,transID_alt,transID_alt_2
0,ENST00000007735.3,3476,ENST00000576236.1
1,ENST00000013070.10,142,ENST00000619583.3
2,ENST00000176183.5,3002,ENST00000611962.2
3,ENST00000203664.9,139,ENST00000617748.3
4,ENST00000211379.9,392,ENST00000415345.5
5,ENST00000211379.9,393,ENST00000434446.5
6,ENST00000211379.9,394,ENST00000412012.5
7,ENST00000211413.9,1120,ENST00000419655.6
8,ENST00000211413.9,1121,ENST00000451329.6
9,ENST00000211413.9,1122,ENST00000414687.6


In [12]:
dfT=dfM[dfM["type"]=="transcript"].reset_index(drop=True)
dfT=dfT.merge(mapDF,how='left',left_on='transID',right_on='uid')
dfT['geneID_alt']="CHS."+dfT.qName.str.split(".",expand=True)[1]
perfectMatch["geneID_alt"]="CHS."+perfectMatch.transID_alt_2.str.split(".",expand=True)[1]
perfectMatch["geneID_prim"]="CHS."+perfectMatch.transID_prim.str.split(".",expand=True)[1]
dfT=dfT[dfT['geneID_alt'].isin(set(perfectMatch['geneID_alt']))].reset_index(drop=True)
dfT=dfT.merge(perfectMatch[["geneID_alt",'geneID_prim']],how='left',left_on='geneID_alt',right_on='geneID_alt')
dfT[["qName",'geneID_prim']].to_csv("./prepAnnotation/res_blat/additionalTrans_blat.csv",index=False)
dfT[["qName",'geneID_prim']]

,qName,geneID_prim
0,ENST00000630761.2,CHS.5
1,ENST00000630761.2,CHS.7
2,ENST00000630761.2,CHS.6
3,ENST00000630761.2,CHS.6
4,ENST00000630761.2,CHS.3
5,ENST00000630761.2,CHS.10
6,ENST00000630761.2,CHS.6
7,ENST00000630761.2,CHS.6
8,ENST00000630761.2,CHS.6
9,ENST00000630761.2,CHS.6


In [13]:
perfm = len(set(perfectMatch["transID_alt_2"]))
one_exon = len(set(dfT["qName"]))
tot_query = len(set(dfAll["qName"]))
tot = len((set(perfectMatch["transID_alt_2"]).union(set(dfT["qName"]))).intersection(set(dfAll["qName"])))
print("number of transcript with matching intron structure: "+str(perfm))
print("number of one-exon transcripts which are contained in another transcripts: "+str(one_exon))
print("total number of query transcripts is: "+str(tot_query))
print("total number of perfect transcripts: "+str(tot))

number of transcript with matching intron structure: 2971
number of one-exon transcripts which are contained in another transcripts: 3816
total number of query transcripts is: 3838
total number of perfect transcripts: 3816


In [ ]:
# now need to update after all of blat is done

# after blat is done, if anything else is left - need to then investigate the remainder